In [1]:
import sys
sys.path.append("../")
from thesaurus_parsing.thesaurus_parser import ThesaurusParser
from syntax_tree import SyntaxTree
from os.path import join
import os
import json
from tqdm import tqdm_notebook as tqdm
from collections import Counter

### Определение паттернов гиперонимии

Воспользуемся тем, что теперь все данные лежат в удобном формате. Построим тезаурус и будем искать паттерны гиперонимии.

Сначала прости считаем имеющийся список файлов с обработанными текстами, которые имеются на данный момент (пока что идёт обработка 36к документов)

In [2]:
DIR_PATH = "/home/loginov-ra/MIPT/HypernymyDetection/data/Lenta/texts_tagged_processed_tree"

In [3]:
file_list = os.listdir(DIR_PATH)
file_list = [join(DIR_PATH, filename) for filename in file_list]

In [4]:
len(file_list)

36437

In [5]:
file_list[0]

'/home/loginov-ra/MIPT/HypernymyDetection/data/Lenta/texts_tagged_processed_tree/20100603protes.txt_processed.json'

В каждом файле известен набор синтаксических деревьев и лемматизированных предложений. Загрузим словарь гиперонимов и посмотрим, как проверять в нём наличие отношения гиперонимии между двумя заданными сущностями.

In [6]:
thesaurus = ThesaurusParser("../data/RuThes", need_closure=False, verbose=True)

In [7]:
thesaurus.hypernyms_dict['человек']

['макроорганизм',
 'организм',
 'живой организм',
 'особь',
 'существо',
 'живность',
 'индивидуум',
 'биологический',
 'биологический организм',
 'субъект деятельность',
 'живое',
 'живой существо']

Будем проходить по каждому предложению, искать сущности, которые являются гиперонимами и строить синтаксический паттерн из записанного в данных дерева. По каждому паттерну заведём статистику встречаемости.

**Важно.** Эту процедуру надо производить только на обучающей части данных.

In [8]:
def is_hyponym_hypernym(hypo_cand, hyper_cand):
    if hypo_cand not in thesaurus.hypernyms_dict:
        return False
    return hyper_cand in thesaurus.hypernyms_dict[hypo_cand]

In [9]:
def get_hypernymy_pairs(multitokens):
    pairs = []
    for i, hypernym_candidate in enumerate(multitokens):
        for j, hyponym_candidate in enumerate(multitokens):
            if i == j:
                continue
            if is_hyponym_hypernym(hyponym_candidate, hypernym_candidate):
                pairs.append((j, i))
    return pairs

In [11]:
pattern_counter = Counter()

no_deeppavlov = 0
tree_failures = 0

for filename in tqdm(file_list):
    with open(filename, encoding='utf-8') as sentences_file:
        sentences = json.load(sentences_file)
        for sent in sentences:
            if 'deeppavlov' not in sent:
                no_deeppavlov += 1
                continue
            
            multitokens, main_pos = sent['multi']
            lemmas = sent['deeppavlov']
            pos = sent['pos']
            tree_info = sent['syntax']
            
            tree = SyntaxTree(empty=True)
            tree.load_from_json(tree_info)
            
            for pattern_pair in get_hypernymy_pairs(multitokens):
                hypo_multi, hyper_multi = pattern_pair
                hypo_main, hyper_main = main_pos[hypo_multi], main_pos[hyper_multi]
                try:
                    pattern = tree.get_syntax_pattern(hypo_main, hyper_main, pos, lemmas)
                except:
                    tree_failures += 1
                    continue
                    
                if pattern is None:
                    tree_failures += 1
                    continue
                pattern = ';'.join(pattern)
                pattern_counter[pattern] += 1
                
                #if pattern == '{}:NOUN:conj:NOUN:{}':
                #    print(' '.join(sent['initial']))
                #    print(multitokens[hypo_multi], multitokens[hyper_multi])
                #    print('')

In [18]:
tree_failures

14

In [15]:
pattern_counter.most_common(n=10)

[('{}:NOUN:nmod:NOUN:{}', 4641),
 ('{}:NOUN:appos:NOUN:{}', 1805),
 ('{}:NOUN:conj:NOUN:{};{}:NOUN:cc:CCONJ:и', 1179),
 ('{}:VERB:parataxis:VERB:{}', 1079),
 ('{}:NOUN:conj:NOUN:{}', 1073),
 ('{}:ADJ:amod:NOUN:{}', 1015),
 ('{}:NOUN:obl:VERB:{}', 1000),
 ('{}:VERB:conj:VERB:{}', 987),
 ('и:CCONJ:cc:NOUN:{};{}:NOUN:conj:NOUN:{}', 985),
 ('{}:VERB:advcl:VERB:{}', 842)]

In [16]:
popular_patterns = [pattern[0] for pattern in pattern_counter.most_common(500)]

In [17]:
with open('../data/popular_patterns.csv', 'w') as pattern_file:
    pattern_file.write('\n'.join(popular_patterns))

Видно, что паттерны похожи на правду, и они более адекватные, чем те, которые были раньше. Однако самые частые из них состоят из всего одного перехода.

Посмотрим на то, какие слова стоят рядом с гипонимом и гиперонимом. Ожидаем увидеть что-то вроде слов `такие`, `другие` или похожие.

In [13]:
hypernym_neighbour_cnt = Counter()
hyponym_neighbour_cnt = Counter()
both_neighbour_cnt = Counter()

def add_neighbours(lemmas, ind, ctr):
    if ind > 0:
        ctr[lemmas[ind - 1]] += 1
    if ind + 1 < len(lemmas):
        ctr[lemmas[ind + 1]] += 1

for filename in tqdm(file_list):
    with open(filename, encoding='utf-8') as sentences_file:
        sentences = json.load(sentences_file)
        for sent in sentences:
            if 'deeppavlov' not in sent:
                no_deeppavlov += 1
                continue
            
            multitokens, main_pos = sent['multi']
            lemmas = sent['deeppavlov']
            pos = sent['pos']
            tree_info = sent['syntax']
            
            tree = SyntaxTree(empty=True)
            tree.load_from_json(tree_info)
            
            for pattern_pair in get_hypernymy_pairs(multitokens):
                hypo_multi, hyper_multi = pattern_pair
                hypo_main, hyper_main = main_pos[hypo_multi], main_pos[hyper_multi]
                add_neighbours(lemmas, hypo_main, hyponym_neighbour_cnt)
                add_neighbours(lemmas, hyper_main, hypernym_neighbour_cnt)
                add_neighbours(lemmas, hypo_main, both_neighbour_cnt)
                add_neighbours(lemmas, hyper_main, both_neighbour_cnt)

KeyboardInterrupt: 

In [63]:
both_neighbour_cnt.most_common(n=10)

[(',', 3503),
 ('в', 2618),
 ('"', 1420),
 ('и', 1254),
 ('.', 1155),
 ('на', 1006),
 ('быть', 790),
 ('с', 743),
 ('не', 555),
 ('по', 520)]

Можно выделить следующие подходящие слова:

`который`
`должный`
`также`
`как`
`тот`
`весь`
`другой`
`такой`
`несколько`
`и`
`или`
`по`
`никакой`
`же`
`ряд`
`а`
`так`
`же`

Однако соседним словом в предложении может быть совершенно удалённая по дереву сущность, поэтому имеет смысл посмотреть на детей и родителя гипонима/гиперонима в синтаксическом дереве.
Эту информацию вполне можно получить также из класса `SyntaxTree`

In [66]:
hypernym_neighbour_cnt = Counter()
hyponym_neighbour_cnt = Counter()
both_neighbour_cnt = Counter()

def add_neighbours(tree, lemma, ind, ctr):
    for child in tree.children[ind]:
        ctr[lemma[child]] += 1

for filename in tqdm(file_list):
    with open(filename, encoding='utf-8') as sentences_file:
        sentences = json.load(sentences_file)
        for sent in sentences:
            if 'deeppavlov' not in sent:
                no_deeppavlov += 1
                continue
            
            multitokens, main_pos = sent['multi']
            lemmas = sent['deeppavlov']
            pos = sent['pos']
            tree_info = sent['syntax']
            
            tree = SyntaxTree(empty=True)
            tree.load_from_json(tree_info)
            
            for pattern_pair in get_hypernymy_pairs(multitokens):
                hypo_multi, hyper_multi = pattern_pair
                hypo_main, hyper_main = main_pos[hypo_multi], main_pos[hyper_multi]
                add_neighbours(tree, lemmas, hypo_main, hyponym_neighbour_cnt)
                add_neighbours(tree, lemmas, hyper_main, hypernym_neighbour_cnt)
                add_neighbours(tree, lemmas, hypo_main, both_neighbour_cnt)
                add_neighbours(tree, lemmas, hyper_main, both_neighbour_cnt)

In [70]:
both_neighbour_cnt.most_common(n=50)

[('.', 3717),
 (',', 3630),
 ('в', 1916),
 ('"', 1100),
 ('и', 947),
 ('быть', 639),
 ('на', 569),
 ('что', 500),
 ('по', 402),
 ('который', 394),
 ('тот', 364),
 ('год', 324),
 ('с', 323),
 ('должный', 311),
 ('не', 308),
 ('-', 283),
 ('один', 259),
 ('как', 253),
 ('он', 236),
 ('это', 226),
 ('мочь', 225),
 ('россия', 211),
 ('заявить', 206),
 ('а', 206),
 ('принять', 205),
 ('из', 195),
 ('этот', 184),
 ('сообщать', 183),
 ('о', 180),
 ('стать', 177),
 ('(', 171),
 (')', 171),
 ('компания', 167),
 ('--', 165),
 ('провести', 160),
 ('получить', 157),
 ('находиться', 136),
 ('к', 134),
 ('для', 131),
 ('процент', 127),
 ('являться', 126),
 ('два', 126),
 ('они', 125),
 ('человек', 123),
 ('решение', 122),
 ('работа', 117),
 ('территория', 111),
 ('система', 111),
 ('страна', 110),
 ('дело', 109)]

**Изменение алгоритма в построении паттернов**

Данные уже почище, но тем не менее новых адекватных слов для добавления нет. Так что алгоритм добавления примерно следующий:

* смотрим слово, от которого требуется построить паттерн

* смотрим на его детей. если среди них есть слово из выделенных, то строим паттерн от него

* проверяем только, что это слово уже не является соседом в пути